# Load

In [ ]:
datadir <- '/n/scratch3/users/i/ik97/'

In [ ]:
suppressWarnings({
    source('utils_celldive.R')    
})


# Label Niches

In [ ]:
spots <- readRDS(glue('{datadir}/lip1_spots.rds'))
cells <- readRDS(glue('{datadir}/lip1_cells.rds'))


## Plot clusters

In [ ]:
## Only plot up to 33 colors 
with(spots, {
    clusters_plot <- names(which(Clusters %>% map(table) %>% map_int(length) <= length(colors_overload)))
    fig.size(4 * length(clusters_plot), 15)
    map(clusters_plot, function(.name) {
        do_scatter(U1$embedding, Clusters, .name) |
        do_scatter(data.frame(x = metadata$y, y = -metadata$x), Clusters, .name, do_labels = FALSE)
    }) %>% 
        reduce(`/`)
    
})


In [ ]:
fig.size(10, 10)
with(spots, do_scatter(U1$embedding, Clusters, 'Clust0.8'))


In [ ]:
fig.size(15, 10)
with(spots, {
    do_scatter(U1$embedding, Clusters, 'Clust0.8', quo(`Clust0.4`), do_labels=FALSE, nrow=6)   
})


## EXTRA: subcluster

In [ ]:
spots$metadata$Cluster_sub <- spots$Clusters$`Clust0.8`
spots$metadata$Cluster_sub <- unlist(do_subcluster(spots, 'U1', .2, spots$metadata$Cluster_sub, c('2')))
spots$metadata$Cluster_sub <- unlist(do_subcluster(spots, 'U1', .3, spots$metadata$Cluster_sub, c('8')))

In [ ]:
library(RColorBrewer)
n <- length(unique(spots$metadata$Cluster_sub))
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
pal = unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))

fig.size(8, 12)
with(spots, {
    do_scatter(U1$embedding, metadata, 'Cluster_sub', do_labels = TRUE, palette_use = pal)
})


In [ ]:
spots$Markers$Cluster_sub <- wilcoxauc(t(spots$z), spots$metadata$Cluster_sub)

## Plot markers (UMAP)

In [ ]:
fig.size(4, 15)
plotFeatures(t(spots$z), spots$U1$embedding, c('ASMA', 'CD31', 'CD90', 'CD146'), nrow = 1,no_guide = TRUE, empty_theme = TRUE) %>% plot()
plotFeatures(t(spots$z), spots$U1$embedding, c('CCL19', 'CD3', 'CD4', 'CD45'), nrow = 1,no_guide = TRUE, empty_theme = TRUE) %>% plot() 
plotFeatures(t(spots$z), spots$U1$embedding, c('VIM', 'PANCK', 'PDPN', 'SPARC', 'PDGFRA'), nrow = 1, no_guide = TRUE, empty_theme = TRUE) %>% plot() 



## Plot markers (heatmap)

In [ ]:
fig.size(4, 12)
# plot_heatmap(spots$Markers$`Clust0.1`, c('ASMA', 'CD31', 'CD90', 'CD146', 'CCL19', 'CD3', 'CD4', 'CD45', 'VIM', 'PANCK'), TRUE)
plot_heatmap(spots$Markers$Cluster_sub, c('ASMA', 'CD31', 'CD146', 'CD3', 'PANCK'), TRUE)
# plot_heatmap(spots$Markers$`Clust0.8`, c('ASMA', 'CD31', 'CD146', 'CD3', 'PANCK'), TRUE)



## Label 

In [ ]:
if ('Niche' %in% colnames(spots$metadata)) 
    spots$metadata$Niche <- NULL

spots$metadata$Cluster <- paste0('C', spots$metadata$Cluster_sub) ## subclustering of Clust0.8
# spots$metadata$Cluster <- paste0('C', spots$Clusters$`Clust0.8`)

spots$metadata <- spots$metadata %>% 
    dplyr::mutate(Niche = case_when(
        Cluster %in% paste0('C', c(13)) ~ 'Lymphoid1',
        Cluster %in% paste0('C', c(22)) ~ 'Lymphoid2',
        # Cluster %in% paste0('C', c()) ~ 'Lymphoid3',
        # Cluster %in% paste0('C', c(12)) ~ 'Garbage',
        Cluster %in% paste0('C', c(3)) ~ 'Perivascular2',
        # Cluster %in% paste0('C', c(8)) ~ 'Perivascular3',
        Cluster %in% paste0('C', c('8_0', '8_8', '8_6')) ~ 'Perivascular3',
        Cluster %in% paste0('C', c('8_3', '8_5', '8_7', '8_4', '8_1', '8_2')) ~ 'Perivascular4',     
        # Cluster %in% paste0('C', c('8_3', '8_5', '8_7', '8_4', '8_1', '8_2')) ~ 'Perivascular4',        

        
        # Cluster %in% paste0('C', c(2)) ~ 'Lympho_Vascular1',
        Cluster %in% paste0('C', c('2_0', '2_5', '2_3')) ~ 'Lymphoid2',
        Cluster %in% paste0('C', c('2_1', '2_2')) ~ 'Lympho_Vascular1',
        
        
        Cluster %in% paste0('C', c(1)) ~ 'Lympho_Vascular2',
        # Cluster %in% paste0('C', c()) ~ 'Lympho_Vascular3',
        # Cluster %in% paste0('C', c()) ~ 'Lympho_Vascular4',
        # Cluster %in% paste0('C', c()) ~ 'Epithelial',
        Cluster %in% paste0('C', c(5)) ~ 'Epithelial_Vascular1',
        # Cluster %in% paste0('C', c()) ~ 'Epithelial_Vascular2',
        # Cluster %in% paste0('C', c()) ~ 'Epithelial_Vascular3',
        # Cluster %in% paste0('C', c()) ~ 'Epithelial_Vascular4',
        # Cluster %in% paste0('C', c()) ~ 'Epithelial_Vascular5',
        TRUE ~ 'Other'
    )) %>% 
    dplyr::mutate(Niche_Broad = gsub('\\d+', '', Niche))


## Viz Labels (UMAP)

In [ ]:
fig.size(8, 12)
with(spots, {
    do_scatter(U1$embedding, metadata, 'Niche', do_labels = TRUE)    
})


## Viz Labels (Physical)

In [ ]:
fig.size(8, 12)
with(spots, {
    do_scatter(
        data.frame(x = metadata$y, y = -metadata$x),
        metadata, 
        'Niche', quo(Niche_Broad), 
        do_labels = FALSE, 
        nrow = 2
    )    
})


## Niche heatmap

In [ ]:
fig.size(8, 10)
with(spots, wilcoxauc(t(z), metadata$Niche)) %>% 
    plot_heatmap(.scale=FALSE)

## Cache

In [ ]:
saveRDS(spots, glue('{datadir}/lip1_spots.rds'))

# Fibroblast fine-grained

In [ ]:
cells <- readRDS(glue('{datadir}/lip1_cells.rds'))


## Visualize cell clusters

In [ ]:
fig.size(6, 8)
with(cells, do_scatter(U1$embedding, Clusters, 'Clust0.8'))


In [ ]:
fig.size(10, 15)
with(cells, {
    plotFeatures(t(z), U1$embedding, c('ASMA', 'CD31', 'CD90', 'PDGFRA', 'CD146', 'PDPN', 'PANCK', 'CD45', 'CD3', 'SPARC', 'CD68', 'CCL19'), nrow = 3, no_guide = TRUE, empty_theme = TRUE) %>% plot()
})


In [ ]:
spots <- readRDS(glue('{datadir}/lip1_spots.rds'))
cells$metadata$Niche <- NULL
cells$metadata$Niche <- cells$metadata %>% 
    left_join(
        dplyr::select(spots$metadata, SpotID, Niche), 
        by = c('CellID' = 'SpotID')
    ) %>% 
    with(Niche)


In [ ]:
fig.size(8, 12)
with(cells, do_scatter(U1$embedding, metadata, 'Niche', quo(Niche), nrow=3, do_labels = FALSE))


## Isolate fibroblasts

In [ ]:
fig.size(5, 12)
p1 <- data.table(cbind(cells$metadata, cells$z, Cluster=cells$Clusters$`Clust0.8`))[
    , V1 := median(PDPN), by = Cluster
] %>% 
    tidyr::gather(key, val, PDPN, PDGFRA, CD90) %>% 
    ggplot(aes(val, reorder(Cluster, V1), color = key)) + 
    # ggplot(aes(PDPN + PDGFRA, reorder(Cluster, V1))) + 
        geom_density_ridges2(fill = NA) + 
        scale_color_tableau() + 
        geom_vline(xintercept = c(0), linetype = 2) + 
        NULL 

p2 <- cells$z %>% 
    data.frame() %>% 
    dplyr::select(PDPN, PDGFRA) %>% 
    do_scatter(cbind(cells$Clusters, cells$metadata), 'Clust0.8', do_labels = FALSE, no_guides = FALSE) + 
        labs(x = 'PDPN', y = 'PDGFRA') + 
        facet_wrap(~LibraryID, scales = 'free') + 
        geom_vline(xintercept = 1, linetype = 2) + 
        geom_hline(yintercept = -.2, linetype = 2) + 
        NULL

(p1 | p2) + plot_layout(widths = c(1, 2.5))


In [ ]:
fig.size(6, 8)
plot_biaxial(cells$z, 'PDPN', 'PDGFRA', 1.5, 0)

In [ ]:
fibroblast_clusters <- cells$Markers$`Clust0.4` %>% 
    subset(
        (feature %in% c('PDPN') & auc > .6) | 
        (feature %in% c('PDGFRA') & auc > .6)
    ) %>% 
    with(unique(group))

message('Fibroblast clusters:')
fibroblast_clusters

idx <- cbind(Cluster = cells$Clusters$`Clust0.4`, cells$z) %>% 
    data.frame() %>% 
    tibble::rowid_to_column('id') %>% 
    subset(Cluster %in% fibroblast_clusters) %>% 
    subset(PDPN > 1.5 | PDGFRA > 0.5) %>% 
    with(id)

message('Number of fibroblasts:')
length(idx)

In [ ]:
fig.size(6, 8)
with(cells, do_scatter(
    U1$embedding, 
    tibble(val = seq_len(nrow(cells$metadata)) %in% idx), 
    'val', 
    quo(val)
))


In [ ]:
fibroblasts <- list(
    metadata = cells$metadata[idx, ], 
    intensity = cells$intensity[idx, ],
    Clusters = cells$Clusters[idx, ]
)


## PCA etc. 

In [ ]:
system.time({
     fibroblasts <- fibroblasts %>% 
        do_norm() %>% 
        do_scale(3, TRUE) %>% 
        do_pca() %>% 
        do_umap('V', 'U1') %>% 
        do_louvain('U1', c(.1, .4, .8, 1.2)) %>% 
        do_markers() %>% 
        identity()
})


In [ ]:
fig.size(6, 12)
with(fibroblasts, do_scatter(U1$embedding, Clusters, 'Clust0.4')) | 
with(fibroblasts, do_scatter(U1$embedding, Clusters, 'Clust0.8'))


## Viz markers

In [ ]:
fig.size(4, 15)
with(fibroblasts, {
    plotFeatures(t(z), U1$embedding, c('SPARC', 'CD90', 'CD31', 'CD146', 'PDPN'), nrow = 1, no_guide = TRUE, empty_theme = TRUE) %>% plot()
    plotFeatures(t(z), U1$embedding, c('CCL19', 'HLADR', 'CD3', 'CD45', 'PANCK'), nrow = 1, no_guide = TRUE, empty_theme = TRUE) %>% plot()
    # plotFeatures(t(z), U1$embedding, c('CD3', 'CCL19', 'HLADR', 'CD90', 'SPARC', 'PDPN', 'CD68', 'CD31'), nrow = 2, no_guide = TRUE, empty_theme = TRUE) %>% plot()
})



In [ ]:
plot_heatmap(fibroblasts$Markers$`Clust0.4`, c('CCL19', 'PDPN', 'PDGFRA', 'SPARC', 'CD90'))

## Viz Niches

In [ ]:
# spots <- readRDS(glue('{datadir}/lip1_spots.rds'))
fibroblasts$metadata$Niche <- fibroblasts$metadata %>% 
    dplyr::select(-Niche) %>% 
    left_join(
        dplyr::select(spots$metadata, SpotID, Niche), 
        by = c('CellID' = 'SpotID')
    ) %>% 
    with(Niche)


In [ ]:
fig.size(8, 12)
with(fibroblasts, do_scatter(U1$embedding, metadata, 'Niche', quo(Niche), nrow=3, do_labels = FALSE))


## Label subtypes 

In [ ]:
fig.size(5, 12)
plot_biaxial(fibroblasts$z, 'CCL19', 'CD90', 1, 1) | 
plot_biaxial(fibroblasts$z, 'SPARC', 'CD90', 1, 1) 


In [ ]:
fibroblasts$metadata$Subtype <- data.frame(fibroblasts$z) %>% 
    cbind(fibroblasts$metadata) %>% 
    cbind(fibroblasts$Clusters) %>% 
    dplyr::mutate(Subtype = case_when(
        `Clust0.4` %in% c('7', '1') & CCL19 > 1 ~ 'Immuno',
        `Clust0.4` %in% c('3', '4') & (SPARC > 0 & CD90 > 0) ~ 'Vascular',
        # `Clust0.4` %in% c('3', '4') & (SPARC > 1 | CD90 > 1) ~ 'Vascular',
        # CCL19 > 1 ~ 'Immuno',
        # SPARC > 1 ~ 'Vascular',
        TRUE ~ 'Fibroblast'
    )) %>% 
    with(Subtype)
table(fibroblasts$metadata$Subtype)


In [ ]:
fig.size(6, 15)
with(fibroblasts, do_scatter(U1$embedding, metadata, 'Subtype', no_guides=FALSE, do_labels=FALSE)) | 
with(fibroblasts, do_scatter(U1$embedding, metadata, 'Niche', do_labels = FALSE))



## Cache

In [ ]:
saveRDS(fibroblasts, glue('{datadir}/lip1_fibroblasts.rds'))

In [ ]:
# fibroblasts <- readRDS(glue('{datadir}/lip1_fibroblasts.rds'))

# Co-localization enrichment 

## Rename niches with nice names

In [ ]:
spots <- readRDS(glue('{datadir}/lip1_spots.rds'))
spots$metadata$Niche_nice <- NULL
spots$metadata <- spots$metadata %>% 
    left_join(
        tribble(
            ~Niche, ~Niche_nice,
            'Other', 'Other',
            'Perivascular4', 'Mural', 
            'Perivascular3', 'Vascular',
            'Lympho_Vascular2', 'Other', 
            'Perivascular2', 'Other', 
            'Epithelial_Vascular1', 'Other', 
            'Lymphoid2', 'Lymphoid', 
            'Lympho_Vascular1', 'Lympho_Vascular',
            'Lymphoid1', 'Lymphoid'  

        )
    )



## Attach to fibroblasts

In [ ]:
fibroblasts <- readRDS(glue('{datadir}/lip1_fibroblasts.rds'))
fibroblasts$metadata$Niche <- NULL
fibroblasts$metadata$Niche_nice <- NULL
fibroblasts$metadata <- fibroblasts$metadata %>% 
    left_join(
        dplyr::select(spots$metadata, SpotID, Niche, Niche_nice), 
        by = c('CellID' = 'SpotID')
    ) 


## Nice co-loc plots

In [ ]:
fig.size(8, 20)
nice_plot_coloc(spots, fibroblasts, c('Mural'), 'Vascular')


In [ ]:
fig.size(8, 20)
nice_plot_coloc(spots, fibroblasts, c('Lymphoid'), 'Immuno')


## Enrichment

In [ ]:
get_coloc_stats <- function(obj) {
    X <- with(obj$metadata, table(Niche_nice, Subtype)) %>% data.frame() 
    res <- expand.grid(Niche = unique(X$Niche_nice), Type = unique(X$Subtype)) %>% apply(1, function(vals) {
        glm(
            y ~ 1 + x, 
            family = poisson, 
            X %>% dplyr::mutate(y = Subtype==vals[['Type']], x = Niche_nice == vals[['Niche']]),
            weights = Freq
        ) %>% 
            broom::tidy() %>% 
            subset(term == 'xTRUE') %>% 
            dplyr::mutate(Niche_nice = vals[['Niche']], Subtype=vals[['Type']]) %>% 
            dplyr::select(-term) %>% 
            dplyr::select(Niche_nice, Subtype, everything())

    }) %>% 
        bind_rows() 
    return(res)
}

In [ ]:
coloc_heatmap <- function(stats) {
    stats %>% 
        dplyr::select(Niche_nice, Subtype, estimate) %>% 
        tidyr::spread(Niche_nice, estimate) %>% 
        tibble::column_to_rownames('Subtype') %>% 
        as.matrix() %>% 
        Heatmap(column_names_rot = 45)
    
}



In [ ]:
res_coloc <- get_coloc_stats(fibroblasts)
res_coloc %>% arrange(-statistic) %>% head(3)

In [ ]:
fig.size(4, 6)
coloc_heatmap(res_coloc)

## Cache

In [ ]:
saveRDS(fibroblasts, glue('{datadir}/lip1_fibroblasts.rds'))
saveRDS(spots, glue('{datadir}/lip1_spots.rds'))
saveRDS(res_coloc, glue('{datadir}/lip1_res_coloc.rds'))
